A facility is indicated by their NPES_ID.  Every facility that discharges into the wastewater is required to have a permit with the EPA about the amount of water they are allowed to discharge, the quality of that discharge and where the discharge happens. 

Each facility has a COUNTY_CODE, CITY, STATE_CODE, and ZIP, as well as a LAT/LON that we can identify them by.  

For every facility, we can identify the industry they are tied to by two CSVs - 'NPDES_NAICS.csv' and 'NPDES_SICS.csv'.  NAICS - very specific descriptions of each industry, SIC = broader categories of industries. 

Facilities have monitoring requirements on the quality of their water, depending on the industry they are in.  They have pollutants they are expected to discharge and have limits set on the amount of pollutants they are allowed to discharge on a regular basis.  They are required to report to the EPA on certain intervals readings from their water discharge that indicate how much water they are discharging and the level of pollutants in that water. 

A facility is required to have a permit with the EPA.  These are reflected in ICIS_PERMITS.csv, which links to ICIS_FACILITIES.csv by the field NPDES_ID = EXTERNAL_PERMIT_NMBR (I think).  Each facility has one permit.  The permit will tell you what water body the facility is discharging into (STATE_WATER_BODY) and whether it's a major or minor permit (a flag that indicates how many pollutants they are discharging).

If a facility violates their permit in some way (and there are over 100 ways they can do this), they are issued a violation.  The outcome of a violation can be nothing, a fine, or an enforcement action, which is a legal mechanism that requires you to do something to fix the violation.  

We can see violations in NPDES_SE_VIOLATIONS.CSV, which links to ICIS_FACILITIES.csv by NPDES_ID.  We can aggregate and find the worst offenders by county. 

However, we most likely want to look at 'NPDES_EFF_VIOLATIONS.CSV'. 

If a facility violates their permit in some way (and there are many different ways), they can receive a Single Event Violation (NPDES_SE_VIOLATIONS.csv).  If they ignore it or it's really bad the first time, the EPA can take some actions.  They take one of two actions: an informal or formal enforcement action.  (NOTE: it has to get really bad to get to either of these actions). 

Informal Enforcment Action: This means they basically say 'Go install this water treatment option', but there is no legal requirement associated with this.  (NPDES_INFORMAL_ENFORCEMENT_ACTIONS.CSV - links by NPDES_ID)

Formal Enforcement Action: You MUST do this certain action to rectify your violation and there is a legal requirement tied to this demand.  (NPDES_FORMAL_ENFORCEMENT_ACTIONS.csv - links by NPDES_ID)

Finally, there is another class of violations call 'effluent violations'.  An 'effluent violation' means - 'You were only allowed to discharge X amount of Y pollutant and you discharged > X amount in a given period.'  The other violations could be related to many other factors that aren't directly tied to water quality.  (This is in 'NPDES_EFF_VIOLATIONS.csv and this also links to ICIS_FACILITIES.csv by NPDES_ID.)   

Each effluent violation is uniquely indicated by a 'NPDES_VIOLATION_ID'.  If a facility is so bad they don't respond to violations, the EPA can as above, conduct an enforcement action, indicated in NPDES_VIOLATION_ENFORCEMENTS.csv, linked by NPDES_VIOLATION_ID.

FINALLY, the EPA will conduct inspections of facilities that are issued violations or that are suspcected of not following their permits, or potentially just randomly conduct inspections.  We can tie together the amount of inspections a facility receives (and compare to the violations and enforcement actions) by NPDES_INSPECTIONS.csv, which links via NPDES_ID.  We should expect facilities with enforcement actions to be inspected more frequently.  

IMPORTANT: This all state level data aggregated and reported by the EPA.  Individual states are delegated the authority for permitting and enforcing permits so variations that we identify occur at a state level, not necessarily at the EPA level.

QUESTIONS:

What geographic areas have the most EPA violations per facility? Can we segment by industry?

How does the amount of violations in a geographic area predict water quality in that area? 

Given the areas that have the most violations, what areas does the EPA actually engage in enforcement actions in?  Does this vary by income levels or demographics? 

Does the EPA engaging in enforcement actions for high-violation facilities improve the water quality in those geographic areas over time? 

Finally, how predictive is an EPA enforcement action on improving not only water quality, but outcomes for the geographic area in general (test scores, college attainment, income levels, etc)


OVERALL THEME: Are certain states better than others at conducting enforcement actions given violations?  Does this vary by income level within state (i.e. are violations/enforcement actions more likely to occur in rich vs poor areas?) Finally, are enforcement actions actually effective at improving both water quality and outcomes for an area?

In [1]:
import pandas as pd

In [6]:
facilities = pd.read_csv('ICIS_FACILITIES.csv')

#add in more detailed industry description
NAICS = pd.read_csv('NPDES_NAICS.csv')
#facilities = pd.merge(facilities, NAICS[['NPDES_ID', 'NAICS_DESC']], on='NPDES_ID')

#permits allocated to each facility
#permits = pd.read_csv("ICIS_PERMITS.csv")

#non effluent violations (i.e. ya'll fucked up in some way)
#single_event_violations = pd.read_csv("NPDES_SE_VIOLATIONS.csv")
#enforcement actions for non-effluent violations
#informal_enf = pd.read_csv('NPDES_INFORMAL_ENFORCEMENT_ACTIONS.csv')
#formal_enf = pd.read_csv('NPDES_FORMAL_ENFORCEMENT_ACTIONS.csv')
#effluent violations (over permit amount)
#eff_violations = pd.read_csv('NPDES_EFF_VIOLATIONS.csv')
#enforcement for effluent violations
#violations = pd.read_csv('NPDES_VIOLATION_ENFORCEMENTS.csv')

To convert county code to fips code - use the below.  Already did this so only need this if we are going to re-run it.  The ICIS_FACILITIES.CSV should already have this column

In [58]:
dtype_dic= {'STATE' : str,
            'STATEFP': str, 
            'COUNTYFP' : str,
            'COUNTYNAME' : str,
            'CLASSFP' : str}
names=['STATE', 'STATEFP', 'COUNTYFP', 'COUNTYNAME', 'CLASSFP']
df = pd.read_csv('national_county.txt', sep=",", names=names, dtype=str)
df['COUNTY_CODE'] = df['STATE'] + df['COUNTYFP']
df['FIPS_CODE'] = df['STATEFP'] + df['COUNTYFP']
facilities = pd.merge(facilities, df[['FIPS_CODE', 'COUNTY_CODE']], on='COUNTY_CODE')